<a href="https://colab.research.google.com/github/VishnuRathore98/Machine_Learning/blob/master/Dwell_Time_Analysis_with_Computer_Vision_Real_Time_Stream_Processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q pytube supervision opencv-python ultralytics inference

In [17]:
import pytube
import numpy as np
import supervision as sv
import cv2
import ultralytics
from inference import get_model
from datetime import datetime
from google.colab.patches import cv2_imshow

In [3]:
class FPSBasedTimer:
  def __init__(self, fps):
    self.fps = fps
    self.frame_id = 0
    self.tracker_id2frame_id = {}

  def tick(self, detections):
    self.frame_id += 1
    times = []

    for tracker_id in detections.tracker_id:
      self.tracker_id2frame_id.setdefault(tracker_id, self.frame_id)

      start_frame_id = self.tracker_id2frame_id[tracker_id]
      time_duration = (self.frame_id - start_frame_id)/self.fps
      times.append(time_duration)

    return np.array(times)

In [9]:
class ClockBasedTimer:
  def __init__(self):
    self.tracker_id2start_time = {}

  def tick(self, detections):
    current_time = datetime.now()
    times = []

    for tracker_id in detections.tracker_id:
      self.tracker_id2start_time.setdefault(tracker_id, current_time)

      start_time =  self.tracker_id2start_time[tracker_id]
      time_duration = (current_time - start_time).total_seconds()
      times.append(time_duration)

    return np.array(times)

In [13]:
model = get_model(model_id="yolov8x-640")

In [20]:
CONFIDENCE_THRESHOLD = 0.5
IOU_THRESHOLD = 0.5
SOURCE_VIDEO_PATH = "/content/checkout.mp4"
RESULT_VIDEO_PATH = "/content/checkout_result.mp4"
video_info = sv.VideoInfo.from_video_path(SOURCE_VIDEO_PATH)

BOUNDING_BOX_ANNOTATOR = sv.BoxAnnotator()

In [ ]:
frame_generator = sv.get_video_frames_generator(source_path=SOURCE_VIDEO_PATH)
with sv.VideoSink(target_path=RESULT_VIDEO_PATH, video_info=video_info) as sink:
  for frame in frame_generator:
    results = model.infer(frame, confidence=CONFIDENCE_THRESHOLD, iou=IOU_THRESHOLD)[0]
    detections = sv.Detections.from_inference(results)

    annotated_frame = frame.copy()
    annotated_frame = BOUNDING_BOX_ANNOTATOR.annotate(scene = annotated_frame, detections = detections)

    sink.write_frame(annotated_frame)
# cv2_imshow(annotated_frame)
# cv2_destroyAllWindows()